In [3]:
import pandas as pd
import bs4, requests, re

In [137]:
def obter_noticias_gizmodo(pag_inicio=1, pag_fim=2):
    df_noticias = pd.DataFrame()
    
    
    for page in range(pag_inicio, pag_fim):
        try:

            res = requests.get('https://gizmodo.uol.com.br/page/%d/'%page)
            soup_noticias = bs4.BeautifulSoup(
                                    res.content.decode('utf-8','ignore'), 
                                    'lxml'
                            )

            # Obtem os links das noticias da página
            noticias = soup_noticias.select('article h3 a')
            for link_noticia in noticias:
                noticia = obter_noticia(link_noticia)
                df_noticias = df_noticias.append(noticia, ignore_index=True)

        except Exception as e:
            print(e, 'Page: %d'%page)
            break
            
    return df_noticias


def obter_noticia(link):
    noticia = { 'url': link.attrs['href'] }

    res = requests.get(noticia['url'])
    soup_noticia = bs4.BeautifulSoup(
                        res.content.decode('utf-8','ignore'), 
                        'lxml'
                    )

    noticia['titulo'] = soup_noticia.find('h1', itemprop="headline").getText()
    noticia['autor'] = soup_noticia.find('a', rel="author").getText()
    noticia['publicacao'] = soup_noticia.find('abbr', itemprop="datePublished").attrs['content']
    noticia['categoria'] = soup_noticia.find('a', class_="postCategory").getText()

    # Obtendo os paragrafos da noticia
    paragrafos = soup_noticia.find('div', itemprop="articleBody").select('p')
    noticia['texto'] = '\n\n'.join([p.getText() for p in paragrafos])
    
    return noticia


In [166]:
df = obter_noticias_gizmodo(120, 200)

In [167]:
df.shape

(800, 6)

In [168]:
df.head()

,autor,categoria,publicacao,texto,titulo,url
0,Noah Berlatsky,cinema,2018-07-04 11:26:06,O primeiro filme inspirado em um personagem da...,O primeiro filme de super-herói tem mais de 10...,https://gizmodo.uol.com.br/judex-primeiro-film...
1,George Dvorsky,astronomia,2018-07-04 10:34:24,"Obrigado, sonda Juno, por esta gloriosa nova i...",A Juno nos trouxe uma bela nova foto do hemisf...,https://gizmodo.uol.com.br/juno-nova-foto-hemi...
2,Kashmir Hill,smartphone,2018-07-04 10:03:55,Essa é uma teoria da conspiração que não sai d...,Estes acadêmicos testaram por um ano se os cel...,https://gizmodo.uol.com.br/estudo-celular-micr...
3,George Dvorsky,ciência,2018-07-04 08:38:38,Analisar livros antigos não parece algo partic...,Pesquisadores descobriram livros venenosos na ...,https://gizmodo.uol.com.br/livros-venenosos-bi...
4,Guilherme Tagiaroli,Leis,2018-07-03 19:04:29,"Aparentemente, os políticos entenderam a impor...",PL de proteção de dados passa por comissão e a...,https://gizmodo.uol.com.br/lei-protecao-dados-...


In [169]:
df.to_csv('noticias_gizmodo_2.csv', index=False)